In [1]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import california_housing

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import sys
import math

import numpy as np
import math

import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

import numpy as np
import pandas as pd
from scipy.special import logit
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import LeaveOneOut, cross_val_predict, StratifiedKFold
from scipy.stats import norm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def calculate_metrics(model, loader):
    model.eval()
        
    with torch.no_grad():
        outputs = model(loader.val_data_tensor, loader.raw_output_val)
        
        _, preds = torch.max(outputs, 1)
        
        all_preds = preds.cpu().numpy()
        all_labels = loader.val_labels_tensor.cpu().numpy()
    
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return accuracy, f1

In [4]:
class CustomDataLoader:
    def __init__(self, features, labels, batch_size=1, validation_size=0.0, raw_output=None):
        train_indices, val_indices, train_labels, val_labels = train_test_split(
            range(len(labels)), labels, test_size=0.2, stratify=labels, random_state=42
        )
    
        train_data = features[train_indices]
        val_data = features[val_indices]

        self.train_data_tensor = torch.tensor(train_data).float().to(device)
        self.train_labels_tensor = torch.tensor(train_labels).long().to(device)
        
        if raw_output is None:
            self.raw_output_train = None
            self.raw_output_val = None
        else:
            self.raw_output_train = raw_output[train_indices].clone().detach().float().to(device)
            self.raw_output_val = raw_output[val_indices].clone().detach().float().to(device)

        self.val_data_tensor = torch.tensor(val_data).float().to(device)
        self.val_labels_tensor = torch.tensor(val_labels).long().to(device)

        train_dataset = TensorDataset(self.train_data_tensor, self.train_labels_tensor)
        val_dataset = TensorDataset(self.val_data_tensor, self.val_labels_tensor)

        self.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        self.val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    def get_train_loader(self):
        return self.train_loader
    
    def get_val_loader(self):
        return self.val_loader

In [5]:
def evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, batch_size, patience=None):
    best_val_loss, early_val_accuracy, early_train_loss = float('inf'), 0.0, float('inf')
    best_epoch = 0
    patience_counter = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i in range(0, custom_train_loader.train_data_tensor.size(0), batch_size):
            inputs, labels = custom_train_loader.train_data_tensor[i:i + batch_size], custom_train_loader.train_labels_tensor[i:i + batch_size]
            optimizer.zero_grad()
            
            outputs = None
            if custom_train_loader.raw_output_train is None:
                outputs = model(inputs.view(-1, num_features))
            else:
                outputs = model(inputs.view(-1, num_features), custom_train_loader.raw_output_train[i:i + batch_size])
                
            loss = criterion(outputs, labels, model)
            running_loss += loss.item()
    
            loss.backward()
            optimizer.step()

        model.eval()
        running_val_loss = 0.0

        unregularized_criterion=nn.CrossEntropyLoss()
        with torch.no_grad():
            for i in range(0, custom_train_loader.val_data_tensor.size(0), batch_size):
                inputs, labels = custom_train_loader.val_data_tensor[i:i + batch_size], custom_train_loader.val_labels_tensor[i:i + batch_size]
                inputs, labels = inputs.to(device), labels.to(device)
                
                if custom_train_loader.raw_output_train is None:
                    outputs = model(inputs.view(-1, num_features))
                else:
                    outputs = model(inputs.view(-1, num_features), custom_train_loader.raw_output_val[i:i + batch_size])

                val_loss = unregularized_criterion(outputs, labels)
                running_val_loss += val_loss.item()

            avg_val_loss = running_val_loss / (len(custom_train_loader.get_val_loader()))

            val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader)

            if (epoch + 1) % 1 == 0:
                print(f'Epoch {epoch+1}, Loss: {running_loss / (len(custom_train_loader.get_train_loader()))}')
                print(f'Validation Loss: {avg_val_loss}')
                print(f'Val Accuracy: {val_accuracy:.4f}, Val F1-score: {val_f1:.4f}')
                print()

            if patience is None:
                continue
            elif avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                early_val_accuracy = val_accuracy
                early_train_loss = running_loss / len(custom_train_loader.get_train_loader())
                best_epoch = epoch
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f'Early stopping triggered. Best validation loss at epoch {best_epoch+1}: {best_val_loss:.4f}')
                print(f'Val Accuracy: {early_val_accuracy:.4f}')
                print(f'Early Train Loss: {early_train_loss}')
                break

In [6]:
class CustomLoss(nn.Module):
    def __init__(self, criterion, l1_lambda, l2_lambda):
        super(CustomLoss, self).__init__()
        self.criterion = criterion
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

    def forward(self, outputs, labels, model):
        loss = self.criterion(outputs, labels)
        
        l1_norm = sum(p.abs().sum() for name, p in model.named_parameters() if 'bias' not in name)
        l2_norm = sum(p.pow(2.0).sum() for name, p in model.named_parameters() if 'bias' not in name)
        
        loss += self.l1_lambda * l1_norm + self.l2_lambda * l2_norm
        return loss

In [7]:
data = pd.read_csv('/kaggle/input/forest-cover-type-dataset/covtype.csv')
data = data.dropna()

X = data.drop(["Cover_Type"], axis=1)
y = data["Cover_Type"]

X = pd.get_dummies(X, drop_first=True)
for col in X.columns:
    if (X[col] > 0).all():
        X[col] = np.log(X[col])

print(X.shape)

(581012, 54)


In [8]:
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(y_encoded)

[4 4 1 ... 2 2 2]


In [9]:
class CustomLinearLayer(nn.Module):
    def __init__(self, input_size, output_size, init="looks_linear"):
        super(CustomLinearLayer, self).__init__()
        self.linear = nn.Linear(input_size, output_size, bias=True)
        nn.init.zeros_(self.linear.bias)

        if init == "zero":
            nn.init.zeros_(self.linear.weight)
        elif init == "looks_linear":
            if input_size * 2 != output_size:
                raise ValueError("Output size must be twice that of input size")
                
            with torch.no_grad():
                weight = torch.zeros(input_size * 2, input_size)

                for i in range(self.linear.in_features):
                    weight[2 * i, i] = 1
                    weight[2 * i + 1, i] = -1

                self.linear.weight.copy_(weight)
                nn.init.zeros_(self.linear.bias)
                
            """ Example matrix: [
                [1, 0, 0],
                [-1, 0, 0],
                [0, 1, 0],
                [0, -1, 0],
                [0, 0, 1],
                [0, 0, -1]
            ] """

    def forward(self, x):
        return self.linear(x)

In [10]:
class DenseGrowNet(nn.Module):        
    def __init__(self, input_size, output_size, is_first_model):
        super(DenseGrowNet, self).__init__()
        self.activation = nn.ReLU()
        
        self.input_size = input_size
        self.output_size = output_size
        
        self.is_first_model = is_first_model
                        
        if is_first_model:
            self.first_layer = CustomLinearLayer(input_size, output_size, init="zero")
        else:
            self.first_layer = CustomLinearLayer(input_size, input_size * 2, init="looks_linear")
            self.last_layer = CustomLinearLayer(input_size * 2, output_size, init="zero")

    def forward(self, x, prev_output=None):
        if self.is_first_model and prev_output is not None:
            raise ValueError("This is the first model and prev_output is passed in")
        if not self.is_first_model and prev_output is None:
            raise ValueError("This is not the first model and prev_output is not passed in")
        
        if prev_output is None:
            return self.first_layer(x)
        
        x = self.activation(self.first_layer(x))
        return self.last_layer(x) + prev_output
    
    def extract_features(self, x, prev_output=None):
        if self.is_first_model:
            raise ValueError("This is not intended for the first model")
        if not self.is_first_model and prev_output is None:
            raise ValueError("This is not the first model and prev_output is not passed in")
            
        return self.activation(self.first_layer(x))

Remember to experiment with adding the intermediate feature after self.first_layer, which is input_size * 2 amount of features, and see how it goes

In [17]:
num_epochs = 100
num_features = 54
num_classes = 7

model = DenseGrowNet(num_features, num_classes, True).to(device)
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, batch_size=1024, validation_size=0.2, raw_output=None)

criterion = CustomLoss(criterion=nn.CrossEntropyLoss(), l1_lambda=0.0001 * 0, l2_lambda=0.0001 * 0)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for name, param in model.named_parameters():
    break
    print(f"Layer: {name}")
    print(f"Shape: {param.shape}")
    print(param)
    
summary(model, input_size=(1, num_features))
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 1024, 5)

Epoch 1, Loss: 1.318782829371843
Validation Loss: 1.0346913060598206
Val Accuracy: 0.6637, Val F1-score: 0.6545

Epoch 2, Loss: 0.9253420968937979
Validation Loss: 0.8575526387022253
Val Accuracy: 0.6993, Val F1-score: 0.6840

Epoch 3, Loss: 0.8116057231562779
Validation Loss: 0.7859727635718229
Val Accuracy: 0.7096, Val F1-score: 0.6935

Epoch 4, Loss: 0.7591742148483377
Validation Loss: 0.7459838170754282
Val Accuracy: 0.7123, Val F1-score: 0.6964

Epoch 5, Loss: 0.729164342129283
Validation Loss: 0.7230947498689618
Val Accuracy: 0.7134, Val F1-score: 0.6977

Epoch 6, Loss: 0.7108395321516213
Validation Loss: 0.7074939723600421
Val Accuracy: 0.7140, Val F1-score: 0.6986

Epoch 7, Loss: 0.6971926093101501
Validation Loss: 0.694982818867031
Val Accuracy: 0.7147, Val F1-score: 0.6998

Epoch 8, Loss: 0.6859414079903506
Validation Loss: 0.6843836961085337
Val Accuracy: 0.7153, Val F1-score: 0.7007

Epoch 9, Loss: 0.6765750082053802
Validation Loss: 0.675794576866585
Val Accuracy: 0.7159, 

In [20]:
X_tensor = torch.tensor(x_scaled).float().to(device)

x_scaled_1 = None
raw_output = None

with torch.no_grad():
    raw_output = model(X_tensor)
#     raw_output_np = raw_output.detach().numpy()
        
#     x_scaled_1 = np.concatenate((x_scaled, raw_output_np), axis=1)
#     x_scaled_1 = x_scaler.fit_transform(x_scaled_1)

In [24]:
num_features = 54 * 3

model1 = DenseGrowNet(num_features, num_classes, False).to(device)
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, batch_size=1024, validation_size=0.2, raw_output=raw_output)

optimizer = optim.Adam(model1.parameters(), lr=0.001 * 5)
evaluate_model(model1, custom_train_loader, criterion, optimizer, num_epochs, num_features, 1024, patience=5)

Epoch 1, Loss: 0.5568172035632154
Validation Loss: 0.5134862216941097
Val Accuracy: 0.7812, Val F1-score: 0.7778

Epoch 2, Loss: 0.48649787489275576
Validation Loss: 0.4709109606450064
Val Accuracy: 0.8004, Val F1-score: 0.7972

Epoch 3, Loss: 0.45361571283067376
Validation Loss: 0.44528620624751375
Val Accuracy: 0.8135, Val F1-score: 0.8108

Epoch 4, Loss: 0.4332361607955941
Validation Loss: 0.4314359544139159
Val Accuracy: 0.8203, Val F1-score: 0.8178

Epoch 5, Loss: 0.41904814167169746
Validation Loss: 0.41939975059868995
Val Accuracy: 0.8263, Val F1-score: 0.8239

Epoch 6, Loss: 0.4093645595769
Validation Loss: 0.41082171755924557
Val Accuracy: 0.8304, Val F1-score: 0.8279

Epoch 7, Loss: 0.4002596802911044
Validation Loss: 0.40286465146039663
Val Accuracy: 0.8348, Val F1-score: 0.8324

Epoch 8, Loss: 0.39397742550015974
Validation Loss: 0.3968473583982702
Val Accuracy: 0.8385, Val F1-score: 0.8361

Epoch 9, Loss: 0.386856999363143
Validation Loss: 0.3908978342487101
Val Accuracy: 

In [25]:
evaluate_model(model1, custom_train_loader, criterion, optimizer, num_epochs, num_features, 1024, patience=5)

Epoch 1, Loss: 0.31408326143997883
Validation Loss: 0.3222759964696148
Val Accuracy: 0.8699, Val F1-score: 0.8684

Epoch 2, Loss: 0.314072181409151
Validation Loss: 0.32378204219173967
Val Accuracy: 0.8699, Val F1-score: 0.8684

Epoch 3, Loss: 0.3138845597726133
Validation Loss: 0.32059848935980545
Val Accuracy: 0.8702, Val F1-score: 0.8687

Epoch 4, Loss: 0.3136335075689307
Validation Loss: 0.32308152840848553
Val Accuracy: 0.8701, Val F1-score: 0.8686

Epoch 5, Loss: 0.3135193367098922
Validation Loss: 0.32133963024407103
Val Accuracy: 0.8703, Val F1-score: 0.8689

Epoch 6, Loss: 0.3133341419933126
Validation Loss: 0.32164109720472706
Val Accuracy: 0.8704, Val F1-score: 0.8689

Epoch 7, Loss: 0.3133663713013023
Validation Loss: 0.3210052748521169
Val Accuracy: 0.8704, Val F1-score: 0.8689

Epoch 8, Loss: 0.3128247242010638
Validation Loss: 0.320959508157613
Val Accuracy: 0.8707, Val F1-score: 0.8692

Early stopping triggered. Best validation loss at epoch 3: 0.3206
Val Accuracy: 0.87

In [28]:
X_tensor = torch.tensor(x_scaled).float().to(device)

x_scaled_1 = None

with torch.no_grad():
    raw_output_1 = model1(X_tensor, raw_output)
    extracted_features = model1.extract_features(X_tensor, raw_output)
    extracted_features_np = extracted_features.detach().cpu().numpy()
        
    x_scaled_1 = np.concatenate((x_scaled, extracted_features_np), axis=1)
    x_scaled_1 = x_scaler.fit_transform(x_scaled_1)

print(raw_output_1.shape)
print(x_scaled_1.shape)

torch.Size([581012, 7])
(581012, 162)


In [30]:
num_features = 54 * 3

model2 = DenseGrowNet(num_features, num_classes, False).to(device)
custom_train_loader = CustomDataLoader(x_scaled_1, y_encoded, batch_size=1024, validation_size=0.2, raw_output=raw_output_1)

optimizer = optim.Adam(model2.parameters(), lr=0.001 * 5)
evaluate_model(model2, custom_train_loader, criterion, optimizer, num_epochs, num_features, 1024, patience=10)

Epoch 1, Loss: 0.2909118722845279
Validation Loss: 0.301105709713802
Val Accuracy: 0.8794, Val F1-score: 0.8786

Epoch 2, Loss: 0.2869236870024698
Validation Loss: 0.29788085112446233
Val Accuracy: 0.8803, Val F1-score: 0.8795

Epoch 3, Loss: 0.2827115014917525
Validation Loss: 0.293214291857
Val Accuracy: 0.8820, Val F1-score: 0.8813

Epoch 4, Loss: 0.2779341020707517
Validation Loss: 0.2868073437558977
Val Accuracy: 0.8857, Val F1-score: 0.8851

Epoch 5, Loss: 0.2720368458245294
Validation Loss: 0.28007556484979496
Val Accuracy: 0.8883, Val F1-score: 0.8878

Epoch 6, Loss: 0.26521150219676776
Validation Loss: 0.2722932202251334
Val Accuracy: 0.8915, Val F1-score: 0.8911

Epoch 7, Loss: 0.2576635145799704
Validation Loss: 0.2661041982079807
Val Accuracy: 0.8934, Val F1-score: 0.8930

Epoch 8, Loss: 0.25028925507210426
Validation Loss: 0.2596034129199229
Val Accuracy: 0.8963, Val F1-score: 0.8959

Epoch 9, Loss: 0.24362963441316252
Validation Loss: 0.25225018056338294
Val Accuracy: 0.8

In [31]:
X_tensor_1 = torch.tensor(x_scaled_1).float().to(device)

x_scaled_2 = None

with torch.no_grad():
    raw_output_2 = model2(X_tensor_1, raw_output_1)
    extracted_features_1 = model2.extract_features(X_tensor_1, raw_output_1)
    extracted_features_np_1 = extracted_features_1.detach().cpu().numpy()
        
    x_scaled_2 = np.concatenate((x_scaled_1, extracted_features_np_1), axis=1)
    x_scaled_2 = x_scaler.fit_transform(x_scaled_2)

print(raw_output_2.shape)
print(x_scaled_2.shape)

torch.Size([581012, 7])
(581012, 486)


In [35]:
num_features = 54 * 3 * 3

model3 = DenseGrowNet(num_features, num_classes, False).to(device)
custom_train_loader = CustomDataLoader(x_scaled_2, y_encoded, batch_size=1024, validation_size=0.2, raw_output=raw_output_2)


optimizer = optim.Adam(model3.parameters(), lr=0.001 * 0.5)
evaluate_model(model3, custom_train_loader, criterion, optimizer, num_epochs, num_features, 1024, patience=5)

Epoch 1, Loss: 0.12227323400225934
Validation Loss: 0.15077924218616987
Val Accuracy: 0.9424, Val F1-score: 0.9424

Epoch 2, Loss: 0.11484885448795058
Validation Loss: 0.14812028767508373
Val Accuracy: 0.9435, Val F1-score: 0.9434

Epoch 3, Loss: 0.11232224891233024
Validation Loss: 0.14669357429732355
Val Accuracy: 0.9438, Val F1-score: 0.9437

Epoch 4, Loss: 0.11055804903943107
Validation Loss: 0.1454825397384794
Val Accuracy: 0.9444, Val F1-score: 0.9444

Epoch 5, Loss: 0.10905425344072775
Validation Loss: 0.14460988035588934
Val Accuracy: 0.9449, Val F1-score: 0.9448

Epoch 6, Loss: 0.10770429311410445
Validation Loss: 0.14368669832484765
Val Accuracy: 0.9452, Val F1-score: 0.9451

Epoch 7, Loss: 0.10647899947962047
Validation Loss: 0.14290109041490054
Val Accuracy: 0.9456, Val F1-score: 0.9455

Epoch 8, Loss: 0.1051998666696874
Validation Loss: 0.1421888280463846
Val Accuracy: 0.9457, Val F1-score: 0.9456

Epoch 9, Loss: 0.10409912680345484
Validation Loss: 0.14167337302576033
Val

In [36]:
X_tensor_2 = torch.tensor(x_scaled_2).float().to(device)

x_scaled_3 = None

with torch.no_grad():
    raw_output_3 = model3(X_tensor_2, raw_output_2)
    extracted_features_2 = model3.extract_features(X_tensor_2, raw_output_2)
    extracted_features_np_2 = extracted_features_2.detach().cpu().numpy()
        
    x_scaled_3 = np.concatenate((x_scaled_2, extracted_features_np_2), axis=1)
    x_scaled_3 = x_scaler.fit_transform(x_scaled_3)

print(raw_output_3.shape)
print(x_scaled_3.shape)

torch.Size([581012, 7])
(581012, 1458)


In [41]:
num_features = 54 * 3 * 3 * 3

model4 = DenseGrowNet(num_features, num_classes, False).to(device)
custom_train_loader = CustomDataLoader(x_scaled_3, y_encoded, batch_size=1024, validation_size=0.2, raw_output=raw_output_3)

model_parameters = filter(lambda p: p.requires_grad, model4.parameters())
num_params = sum([p.numel() for p in model_parameters])

print(f"Number of trainable parameters: {num_params}")

optimizer = optim.Adam(model4.parameters(), lr=0.001 * 0.01)
evaluate_model(model4, custom_train_loader, criterion, optimizer, num_epochs, num_features, 1024, patience=5)

Number of trainable parameters: 4274863
Epoch 1, Loss: 0.06883399549559875
Validation Loss: 0.13014657645110497
Val Accuracy: 0.9555, Val F1-score: 0.9555

Epoch 2, Loss: 0.06641865326378839
Validation Loss: 0.12832427443119518
Val Accuracy: 0.9559, Val F1-score: 0.9559

Epoch 3, Loss: 0.06518845498102352
Validation Loss: 0.12741034772051008
Val Accuracy: 0.9564, Val F1-score: 0.9564

Epoch 4, Loss: 0.06445436558731327
Validation Loss: 0.1268276388577202
Val Accuracy: 0.9567, Val F1-score: 0.9567

Epoch 5, Loss: 0.06393091304285148
Validation Loss: 0.12640772354707383
Val Accuracy: 0.9568, Val F1-score: 0.9568

Epoch 6, Loss: 0.06352853025863611
Validation Loss: 0.1260855320775718
Val Accuracy: 0.9569, Val F1-score: 0.9569

Epoch 7, Loss: 0.06320466021576379
Validation Loss: 0.12582559635241827
Val Accuracy: 0.9570, Val F1-score: 0.9569

Epoch 8, Loss: 0.06292819792183223
Validation Loss: 0.12560973427536196
Val Accuracy: 0.9569, Val F1-score: 0.9569

Epoch 9, Loss: 0.06268586432520251